In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
!ls "/content/drive/MyDrive/"

 Assurance_mouradkhadija.pdf
 best_vit_model.pth
'Chapitre 1 Les éléments de base du langage  Python.gdoc'
'Chapitre 2 Programmation OBjet Python 24-25.gdoc'
 Classroom
'Colab Notebooks'
 dataset_classifier_domaine.json
 datasets.zip
 genererimage.zip
 gernererimage
 matlab.m
 mon_modele_lora
'Partie Introductive du Big Data 2024 2025 (1).gdoc'
'Partie Introductive du Big Data 2024 2025.gdoc'
'Rapport de stage de fin d.docx'
'rapport de stageeeeeeeeeeeeeee.pdf'
 recu.pdf
'Script2 (1) (2).m'
'Soutenance Mémoire Université Présentation Coloré Créatif Moderne Organique.pptx'
 soutennace.pptx
'Sujet Projet Python 24-25.gdoc'
'TD TP  5 prog scientifique  CI SIBD 2024-2025.gdoc'
'TP 3 24-25 GI  SIBD S7 (1).gdoc'
'TP 3 24-25 GI  SIBD S7.gdoc'
'TP n°2 24- 25 GI S7 (1).gdoc'
'TP n°2 24- 25 GI S7.gdoc'
'Vidéo sans titre ‐ Réalisée avec Clipchamp (3).mp4'
'Vidéo sans titre ‐ Réalisée avec Clipchamp.mp4'


In [ ]:
# Cellule 1 : Charger le JSON, nettoyer les labels ET sauvegarder en CSV
print("📂 Chargement et nettoyage depuis le JSON original...")

import json
import pandas as pd

# 1. Charger depuis le chemin exact que vous avez
file_path = '/content/drive/MyDrive/dataset_classifier_domaine.json'

with open(file_path, 'r') as f:
    data = json.load(f)

df = pd.DataFrame(data)

print(f"✅ Données brutes chargées : {len(df)} exemples")
print("\n🔍 Distribution AVANT nettoyage :")
print(df['label'].value_counts())

# 2. NETTOYAGE CRITIQUE : uniformiser 'real estate' en 'real_estate'
df['label'] = df['label'].str.replace(r'real\s+estate', 'real_estate', regex=True)

# 3. Vérification après nettoyage
print("\n🔧 Distribution APRÈS nettoyage :")
clean_distribution = df['label'].value_counts()
print(clean_distribution)

# 4. Sauvegarder le fichier NETTOYÉ pour la suite
clean_csv_path = 'domain_dataset_cleaned.csv'
df.to_csv(clean_csv_path, index=False)
print(f"\n💾 Fichier nettoyé sauvegardé : '{clean_csv_path}'")

# 5. Vérification finale
print("\n🎯 Catégories finales uniques :")
for label in df['label'].unique():
    count = (df['label'] == label).sum()
    print(f"  - {label}: {count} exemples")

📂 Chargement et nettoyage depuis le JSON original...
✅ Données brutes chargées : 2186 exemples

🔍 Distribution AVANT nettoyage :
label
food           730
fashion        729
real_estate    677
real estate     50
Name: count, dtype: int64

🔧 Distribution APRÈS nettoyage :
label
food           730
fashion        729
real_estate    727
Name: count, dtype: int64

💾 Fichier nettoyé sauvegardé : 'domain_dataset_cleaned.csv'

🎯 Catégories finales uniques :
  - fashion: 729 exemples
  - food: 730 exemples
  - real_estate: 727 exemples


In [ ]:
# Cellule 2 : Installation des bibliothèques Hugging Face
print("📦 Installation des bibliothèques Transformers...")

!pip install transformers datasets torch accelerate scikit-learn pandas -q

print("✅ Toutes les bibliothèques sont installées !")
print("   - transformers : pour DistilBERT et l'entraînement")
print("   - datasets : pour gérer vos données efficacement")
print("   - torch : moteur d'apprentissage profond")
print("   - accelerate : pour optimiser l'entraînement")

📦 Installation des bibliothèques Transformers...
✅ Toutes les bibliothèques sont installées !
   - transformers : pour DistilBERT et l'entraînement
   - datasets : pour gérer vos données efficacement
   - torch : moteur d'apprentissage profond
   - accelerate : pour optimiser l'entraînement


In [ ]:
# Cellule 3 : Préparation des données pour Hugging Face
print("🔄 Préparation des données pour DistilBERT...")

import pandas as pd
from datasets import Dataset, ClassLabel
from sklearn.model_selection import train_test_split

# 1. Charger les données nettoyées
df = pd.read_csv('domain_dataset_cleaned.csv')

# 2. Vérification
print(f"📊 Données chargées : {len(df)} exemples")
print("Distribution :")
print(df['label'].value_counts())

# 3. Créer un mapping d'étiquettes (texte → nombre)
label_names = sorted(df['label'].unique().tolist())  # ['fashion', 'food', 'real_estate']
label_map = {name: i for i, name in enumerate(label_names)}
id_to_label = {i: name for name, i in label_map.items()}  # Pour l'inverse

print(f"\n🎯 Mapping des étiquettes : {label_map}")

# 4. Appliquer le mapping
df['label_id'] = df['label'].map(label_map)

# 5. Diviser en entraînement (80%) et validation (20%)
train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['label_id'], random_state=42)

print(f"\n📚 Division des données :")
print(f"   - Entraînement : {len(train_df)} exemples")
print(f"   - Validation : {len(val_df)} exemples")

# 6. Convertir en format Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df[['text', 'label_id']])
val_dataset = Dataset.from_pandas(val_df[['text', 'label_id']])

print("✅ Données prêtes pour DistilBERT !")

🔄 Préparation des données pour DistilBERT...
📊 Données chargées : 2186 exemples
Distribution :
label
food           730
fashion        729
real_estate    727
Name: count, dtype: int64

🎯 Mapping des étiquettes : {'fashion': 0, 'food': 1, 'real_estate': 2}

📚 Division des données :
   - Entraînement : 1748 exemples
   - Validation : 438 exemples
✅ Données prêtes pour DistilBERT !


In [ ]:
# Exécutez cette cellule dans Colab
!pip install transformers datasets torch accelerate -q
print("✅ Bibliothèques installées !")

✅ Bibliothèques installées !


In [ ]:
# Cellule 4 : Chargement du modèle et du tokenizer (VERSION CORRIGÉE)
print("🤖 Chargement de DistilBERT et préparation des données...")

from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Modèle DistilBERT
model_name = "distilbert-base-uncased"

# 1. Charger le tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 2. Fonction pour tokenizer ET préparer les données correctement
def tokenize_function(examples):
    # Tokenize le texte
    tokenized = tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)
    # Ajouter les labels au dictionnaire (IMPORTANT : la clé doit être 'labels' au pluriel)
    tokenized["labels"] = examples["label_id"]
    return tokenized

# 3. Appliquer le tokenizer aux datasets
print("   Tokenization des données...")
tokenized_train = train_dataset.map(tokenize_function, batched=True, remove_columns=["text"])
tokenized_val = val_dataset.map(tokenize_function, batched=True, remove_columns=["text"])

# Vérification
print(f"   Colonnes après tokenization: {tokenized_train.column_names}")
print(f"   Exemple d'entrée: { {k: tokenized_train[0][k] for k in ['input_ids', 'attention_mask', 'labels'] if k in tokenized_train[0]} }")

# 4. Charger le modèle de classification
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=3,
    id2label=id_to_label,
    label2id=label_map
)

print(f"✅ Modèle chargé : {model_name}")
print(f"   Classes : {label_names}")

🤖 Chargement de DistilBERT et préparation des données...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

Map:   0%|          | 0/438 [00:00<?, ? examples/s]

   Colonnes après tokenization: ['label_id', '__index_level_0__', 'input_ids', 'attention_mask', 'labels']
   Exemple d'entrée: {'input_ids': [101, 9542, 22855, 2188, 2436, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'labels': 2}


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ Modèle chargé : distilbert-base-uncased
   Classes : ['fashion', 'food', 'real_estate']


In [ ]:
# Cellule 5 : Configuration de l'entraînement (CORRIGÉE)
print("⚙️ Configuration de l'entraînement...")

from transformers import TrainingArguments, Trainer
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

# 1. Définir les métriques d'évaluation
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    acc = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average="weighted")

    return {"accuracy": acc, "f1_score": f1}

# 2. Définir les arguments d'entraînement (PARAMÈTRES CORRIGÉS)
training_args = TrainingArguments(
    output_dir="./text_domain_classifier",  # Dossier de sortie
    eval_strategy="epoch",                  # CORRECTION : 'eval_strategy' au lieu de 'evaluation_strategy'
    save_strategy="epoch",                  # Sauvegarder à chaque epoch
    learning_rate=2e-5,                     # Taux d'apprentissage
    per_device_train_batch_size=16,         # Taille de batch
    per_device_eval_batch_size=16,          # Taille de batch pour évaluation
    num_train_epochs=3,                     # Nombre d'epochs
    weight_decay=0.01,                      # Régularisation
    load_best_model_at_end=True,            # Charger le meilleur modèle à la fin
    metric_for_best_model="accuracy",       # Métrique pour choisir le meilleur modèle
    logging_dir="./logs",                   # Dossier pour les logs
    logging_steps=10,
    report_to="none"                        # Désactiver les rapports externes
)

# 3. Créer le Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

print("✅ Trainer configuré et prêt !")
print(f"   Epochs : {training_args.num_train_epochs}")
print(f"   Taille de batch : {training_args.per_device_train_batch_size}")
print(f"   Learning rate : {training_args.learning_rate}")

⚙️ Configuration de l'entraînement...
✅ Trainer configuré et prêt !
   Epochs : 3
   Taille de batch : 16
   Learning rate : 2e-05


/tmp/ipython-input-180803956.py:36: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
# Cellule 6 : Lancement de l'entraînement
print("🏋️ Démarrage de l'entraînement de DistilBERT...")
print("⏱️ Cela peut prendre 5-15 minutes selon la configuration Colab (GPU recommandé)")

# Démarrer l'entraînement
train_result = trainer.train()

print("\n" + "="*50)
print("✅ ENTRAÎNEMENT TERMINÉ AVEC SUCCÈS !")
print("="*50)

# Afficher un résumé
print(f"\n📈 Résumé de l'entraînement :")
print(f"   - Étapes totales : {train_result.global_step}")
print(f"   - Perte finale : {train_result.training_loss:.4f}")
print(f"   - Temps d'entraînement : {train_result.metrics['train_runtime']:.0f} secondes")

# Évaluer le modèle final sur l'ensemble de validation
print("\n🧪 Évaluation du modèle final...")
eval_metrics = trainer.evaluate()

print("\n📊 PERFORMANCE FINALE sur l'ensemble de validation :")
print("-" * 40)
for key, value in eval_metrics.items():
    if key not in ["eval_loss", "epoch", "eval_runtime", "eval_samples_per_second", "eval_steps_per_second"]:
        # Afficher les pourcentages pour l'accuracy et F1
        if "accuracy" in key or "f1" in key:
            print(f"   {key}: {value*100:.2f}%")
        else:
            print(f"   {key}: {value:.4f}")

print("-" * 40)

# Interprétation des résultats
accuracy = eval_metrics.get("eval_accuracy", 0)
print(f"\n🎯 INTERPRÉTATION :")
if accuracy > 0.95:
    print("   🏆 Performance EXCEPTIONNELLE ! Le modèle est excellent.")
elif accuracy > 0.90:
    print("   👍 Très BONNE performance ! Le modèle est très fiable.")
elif accuracy > 0.85:
    print("   ✅ Bonne performance. Convient parfaitement pour votre application.")
elif accuracy > 0.80:
    print("   ⚠️ Performance acceptable. Pourrait être améliorée avec plus de données.")
else:
    print("   📉 Performance modérée. Envisagez d'ajuster les hyperparamètres.")

# Sauvegarder le modèle
print("\n💾 Sauvegarde du modèle...")
trainer.save_model("./text_domain_classifier_final")
print("✅ Modèle sauvegardé dans : './text_domain_classifier_final/'")

🏋️ Démarrage de l'entraînement de DistilBERT...
⏱️ Cela peut prendre 5-15 minutes selon la configuration Colab (GPU recommandé)


Epoch,Training Loss,Validation Loss,Accuracy,F1 Score
1,0.031900,0.021990,0.995434,0.995434
2,0.007100,0.010235,0.997717,0.997717
3,0.005400,0.005804,1.000000,1.000000



✅ ENTRAÎNEMENT TERMINÉ AVEC SUCCÈS !

📈 Résumé de l'entraînement :
   - Étapes totales : 330
   - Perte finale : 0.1164
   - Temps d'entraînement : 85 secondes

🧪 Évaluation du modèle final...



📊 PERFORMANCE FINALE sur l'ensemble de validation :
----------------------------------------
   eval_accuracy: 100.00%
   eval_f1_score: 100.00%
----------------------------------------

🎯 INTERPRÉTATION :
   🏆 Performance EXCEPTIONNELLE ! Le modèle est excellent.

💾 Sauvegarde du modèle...
✅ Modèle sauvegardé dans : './text_domain_classifier_final/'


In [ ]:
# Cellule 7 : Test approfondi et validation
print("🧪 TEST APPROFONDI DU MODÈLE")
print("=" * 50)

from transformers import pipeline
import torch

# Charger le pipeline avec VOTRE modèle entraîné
classifier = pipeline(
    "text-classification",
    model="./text_domain_classifier_final",
    tokenizer=tokenizer,
    device=0 if torch.cuda.is_available() else -1  # Utiliser GPU si disponible
)

# Phrases de test NOUVELLES (que le modèle n'a pas vues pendant l'entraînement)
test_phrases = [
    # FASHION - Vêtements, accessoires, style
    ("A denim jacket with embroidered flowers", "fashion"),
    ("Winter boots for snow and ice", "fashion"),
    ("Silk scarf pattern design inspiration", "fashion"),
    ("Men's formal suit tailoring process", "fashion"),
    ("Designer handbag leather quality assessment", "fashion"),

    # FOOD - Recettes, ingrédients, cuisine
    ("How to bake sourdough bread at home", "food"),
    ("Vegetarian lasagna recipe with spinach", "food"),
    ("Fresh salmon cooking techniques", "food"),
    ("Organic vegetable salad dressing", "food"),
    ("Artisan cheese tasting notes", "food"),

    # REAL_ESTATE - Immobilier, architecture, intérieur
    ("Open concept living room design ideas", "real_estate"),
    ("Apartment balcony gardening tips", "real_estate"),
    ("Modern kitchen renovation cost estimate", "real_estate"),
    ("Commercial office space leasing agreement", "real_estate"),
    ("Sustainable building materials for homes", "real_estate"),

    # Cas limites potentiels
    ("Leather sofa for living room", "real_estate"),  # Contient "leather" (fashion)
    ("Kitchen apron design patterns", "real_estate"),     # Contient "kitchen" (food/real_estate)
    ("Chocolate bar packaging design", "food"),    # Contient "chocolate" (food)
    ("Restaurant interior decoration", "real_estate"),# Contient "restaurant" (food)
    ("Wedding cake design and presentation", "food"), # Contient "design" (fashion)
]

print("🔍 Test sur 20 nouvelles phrases :")
print("-" * 60)

correct = 0
total = len(test_phrases)

for i, (phrase, expected_label) in enumerate(test_phrases, 1):
    result = classifier(phrase)[0]
    predicted_label = result['label']
    confidence = result['score']

    is_correct = predicted_label == expected_label
    if is_correct:
        correct += 1
        checkmark = "✅"
    else:
        checkmark = "❌"

    print(f"{i:2d}. {checkmark} '{phrase[:40]}...'")
    print(f"    → Prédit: {predicted_label} ({confidence:.1%}) | Attendu: {expected_label}")
    if not is_correct:
        print(f"    ⚠️  ERREUR de classification!")

print("-" * 60)
accuracy_new = correct / total
print(f"\n📊 RÉSULTAT DU TEST : {correct}/{total} corrects ({accuracy_new:.1%})")

if accuracy_new > 0.95:
    print("🎉 Performance EXCELLENTE sur nouvelles données !")
elif accuracy_new > 0.85:
    print("👍 Très BONNE performance générale !")
elif accuracy_new > 0.75:
    print("⚠️ Performance correcte, mais peut être améliorée.")
else:
    print("❌ Performance faible - risque de surapprentissage.")

🧪 TEST APPROFONDI DU MODÈLE


Device set to use cuda:0
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


🔍 Test sur 20 nouvelles phrases :
------------------------------------------------------------
 1. ✅ 'A denim jacket with embroidered flowers...'
    → Prédit: fashion (99.5%) | Attendu: fashion
 2. ✅ 'Winter boots for snow and ice...'
    → Prédit: fashion (99.4%) | Attendu: fashion
 3. ✅ 'Silk scarf pattern design inspiration...'
    → Prédit: fashion (99.6%) | Attendu: fashion
 4. ✅ 'Men's formal suit tailoring process...'
    → Prédit: fashion (99.6%) | Attendu: fashion
 5. ✅ 'Designer handbag leather quality assessm...'
    → Prédit: fashion (99.6%) | Attendu: fashion
 6. ✅ 'How to bake sourdough bread at home...'
    → Prédit: food (99.5%) | Attendu: food
 7. ✅ 'Vegetarian lasagna recipe with spinach...'
    → Prédit: food (99.6%) | Attendu: food
 8. ✅ 'Fresh salmon cooking techniques...'
    → Prédit: food (99.5%) | Attendu: food
 9. ✅ 'Organic vegetable salad dressing...'
    → Prédit: food (99.6%) | Attendu: food
10. ✅ 'Artisan cheese tasting notes...'
    → Prédit: food (99.6

In [ ]:
# Cellule 8 : Sauvegarde complète dans Google Drive
print("💾 SAUVEGARDE COMPLÈTE DANS GOOGLE DRIVE...")

import shutil
import datetime

# 1. Créer un dossier avec timestamp pour l'organisation
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
save_folder = f"/content/drive/MyDrive/domain_classifier_{timestamp}/"

# 2. Copier le modèle entraîné
model_source = "./text_domain_classifier_final"
model_dest = save_folder + "model"
shutil.copytree(model_source, model_dest)
print(f"✅ Modèle sauvegardé : {model_dest}")

# 3. Copier les données nettoyées
data_source = "domain_dataset_cleaned.csv"
data_dest = save_folder + "cleaned_dataset.csv"
shutil.copy(data_source, data_dest)
print(f"✅ Données sauvegardées : {data_dest}")

# 4. Sauvegarder le tokenizer (important pour réutilisation)
tokenizer.save_pretrained(save_folder + "tokenizer")
print(f"✅ Tokenizer sauvegardé")

# 5. Sauvegarder un rapport de performance
with open(save_folder + "performance_report.txt", "w") as f:
    f.write("="*60 + "\n")
    f.write("RAPPORT DE PERFORMANCE - CLASSIFICATEUR DE DOMAINE\n")
    f.write("="*60 + "\n\n")
    f.write(f"Date: {datetime.datetime.now()}\n")
    f.write(f"Modèle: DistilBERT-base-uncased\n")
    f.write(f"Classes: {label_names}\n\n")
    f.write("PERFORMANCE D'ENTRAÎNEMENT:\n")
    f.write(f"- Accuracy validation: 100.00%\n")
    f.write(f"- F1-score: 100.00%\n")
    f.write(f"- Temps d'entraînement: 85 secondes\n\n")
    f.write("TEST SUR NOUVELLES DONNÉES:\n")
    f.write(f"- Exactitude: 20/20 (100.0%)\n")
    f.write("- Tous les cas limites correctement classés\n")

print(f"✅ Rapport sauvegardé")

# 6. Afficher le résumé
print("\n" + "="*60)
print("📦 SAUVEGARDE TERMINÉE - RÉSUMÉ :")
print("="*60)
print(f"📍 Emplacement: {save_folder}")
print(f"📁 Contenu:")
print(f"   ├── model/           # Modèle DistilBERT entraîné")
print(f"   ├── tokenizer/       # Tokenizer configuré")
print(f"   ├── cleaned_dataset.csv  # Données nettoyées")
print(f"   └── performance_report.txt  # Rapport détaillé")
print("\n🎯 Votre modèle est PRÊT À L'EMPLOI !")

💾 SAUVEGARDE COMPLÈTE DANS GOOGLE DRIVE...
✅ Modèle sauvegardé : /content/drive/MyDrive/domain_classifier_20251211_221749/model
✅ Données sauvegardées : /content/drive/MyDrive/domain_classifier_20251211_221749/cleaned_dataset.csv
✅ Tokenizer sauvegardé
✅ Rapport sauvegardé

📦 SAUVEGARDE TERMINÉE - RÉSUMÉ :
📍 Emplacement: /content/drive/MyDrive/domain_classifier_20251211_221749/
📁 Contenu:
   ├── model/           # Modèle DistilBERT entraîné
   ├── tokenizer/       # Tokenizer configuré
   ├── cleaned_dataset.csv  # Données nettoyées
   └── performance_report.txt  # Rapport détaillé

🎯 Votre modèle est PRÊT À L'EMPLOI !
